## Data Extraction and data wrangling workflow

### Logic / pipline

1. Purple Air 

1.1. Data is extracted manually as csv files and put on /data/purpleair/sample/

1.2. Data wrangling is done by script

More information in the [/data/purpleair/README.md](/data/purpleair/README.md)

purpleair/merge-pa.py script create first step of csv from the specified folder as below.

Two parameters to this script:

*csv*: csv filename to store the data after wrangling

*folder*: the folder where are located the purpleair csv files (one csv by groundstation) 

In [1]:
%run purpleair/merge-pa.py --csv bigtable.csv --folder purpleair/sample

2. Open AQ

2.1. Data was extracted stored into a s3 Amazon storage

2.2. Data wrangling to extract and merge more groundstations into CSV is done by script here as follow:

In [2]:
import pandas as pd
import numpy as np
import s3fs
import json

In [3]:
openAQcsv= 's3://openaqpm25dcarea/data_load_DC/Unsaved/2020/03/21/5f0f7603-af35-4673-b7cd-d1685d04fed7.csv'
df_openAQ = pd.read_csv(openAQcsv)


/home/julien/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2.2.3. parsing date and coordinates column to extract X, Y and UTC datetime

date format is string with json structure as : '{utc=2017-08-11T05:00:00.000Z, local=2017-08-11T00:00:00-05:00}'

coordinates format is string with a json tructure as : '{latitude=38.92185, longitude=-77.013176}'

To be json compliant = sings should be replaced by : signs
Also missing double quote for strings

In [4]:
df_PM25 = df_openAQ[df_openAQ["parameter"] == 'pm25']
len(df_PM25)
df_PM25.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile
6,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,McMillan NCORE,10.0,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=38.92185, longitude=-77.013176}",US,AirNow,government,False
10,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Weiser Neph,19.9,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=44.261696, longitude=-116.97917}",US,AirNow,government,False
14,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Sipayik-ME,6.5,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=44.9636, longitude=-67.0608}",US,AirNow,government,False
19,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Rockville,14.0,µg/m³,Washington-Arlington-Alexandria,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=39.1144, longitude=-77.1069}",US,AirNow,government,False
22,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,HU-Beltsville,8.0,µg/m³,Washington-Arlington-Alexandria,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=39.0553, longitude=-76.8783}",US,AirNow,government,False


In [5]:
def getDateTime(row):
    json_s=row.date.replace("=", ":\"").replace("}","\"}").replace(".000Z",".000Z\"").replace('utc','"utc"').replace('local','"local"')
    #print(json_s)
    date_json = json.loads(json_s)
    return "{} UTC".format(date_json["utc"].replace("T", " ").replace(".000Z", ""))

def getX(row):
    geo_json = json.loads(row.coordinates.replace("=", ":").replace('longitude','"longitude"').replace('latitude','"latitude"'))
    #print(geo_json)
    return geo_json["latitude"]
    
def getY(row):
    geo_json = json.loads(row.coordinates.replace("=", ":").replace('longitude','"longitude"').replace('latitude','"latitude"'))
    return geo_json["longitude"]

df_PM25['datetime']= df_PM25.apply(lambda row: getDateTime(row), axis=1)
df_PM25['x']= df_PM25.apply(lambda row: getX(row), axis=1)
df_PM25['y']= df_PM25.apply(lambda row: getY(row), axis=1)
df_PM25['pm25']=df_PM25['value']
df_PM25['station_id']=df_PM25['attribution']
df_PM25['sensor']='ND'
df_PM25['type']=df_PM25['sourcename']

df_PM25.head()

/home/julien/.local/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/julien/.local/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/julien/.local/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile,datetime,x,y,pm25,station_id,sensor,type
6,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,McMillan NCORE,10.0,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=38.92185, longitude=-77.013176}",US,AirNow,government,False,2017-08-11 05:00:00 UTC,38.921850,-77.013176,10.0,"[{name=US EPA AirNow, url=http://www.airnow.go...",ND,AirNow
10,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Weiser Neph,19.9,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=44.261696, longitude=-116.97917}",US,AirNow,government,False,2017-08-11 05:00:00 UTC,44.261696,-116.979170,19.9,"[{name=US EPA AirNow, url=http://www.airnow.go...",ND,AirNow
14,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Sipayik-ME,6.5,µg/m³,WASHINGTON,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=44.9636, longitude=-67.0608}",US,AirNow,government,False,2017-08-11 05:00:00 UTC,44.963600,-67.060800,6.5,"[{name=US EPA AirNow, url=http://www.airnow.go...",ND,AirNow
19,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,Rockville,14.0,µg/m³,Washington-Arlington-Alexandria,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=39.1144, longitude=-77.1069}",US,AirNow,government,False,2017-08-11 05:00:00 UTC,39.114400,-77.106900,14.0,"[{name=US EPA AirNow, url=http://www.airnow.go...",ND,AirNow
22,"{utc=2017-08-11T05:00:00.000Z, local=2017-08-1...",pm25,HU-Beltsville,8.0,µg/m³,Washington-Arlington-Alexandria,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=39.0553, longitude=-76.8783}",US,AirNow,government,False,2017-08-11 05:00:00 UTC,39.055300,-76.878300,8.0,"[{name=US EPA AirNow, url=http://www.airnow.go...",ND,AirNow


Filter here the ones inside DC metro area

Using the previously parsed X and Y from coordinates latitudes and longitudes

In [6]:
#same as worldpop 
#@todo x and y were inverted

buffer_deg=0.0027
ymax_MetroArea = -76.3881880661718 + buffer_deg
xmax_MetroArea = 39.5284754397557 + buffer_deg
ymin_MetroArea = -77.7356609923511 - buffer_deg
xmin_MetroArea = 38.3969644315365 - buffer_deg
    
geoFilter = (
    (df_PM25["x"] > xmin_MetroArea) &
    (df_PM25["x"] < xmax_MetroArea) &
    (df_PM25["y"] > ymin_MetroArea) &
    (df_PM25["y"] < ymax_MetroArea) 
)

len(df_PM25[geoFilter])

138727

In [7]:
cols =["datetime", "pm25", "type", "sensor", "station_id", "x", "y"]
#df_PM25.columns.tolist()
df_PM25 = df_PM25[geoFilter]
#df_PM25[cols].head()
len(df_PM25[cols])

138727

In [8]:
df_bigtable = pd.read_csv("bigtable.csv")
nb = len(df_bigtable)
df_bigtable = df_bigtable.append(df_PM25[cols])
print("{} records are now {}".format(nb, len(df_bigtable)))
df_bigtable.type.describe()

105054 records are now 243781


count     243781
unique         2
top       AirNow
freq      138727
Name: type, dtype: object

In [9]:
df_bigtable.to_csv("bigtable.csv")

# WorldPop data extract and wrangling

## worlpop data extract 

Done by python script.

Data is downloaded using python script using provided API 

Data is stored as tiff locally but after clipping it using DC metro area geo extent (because full US tiff is above 10Gb by tiff)

## worldpop data wrangling

Using the python script and GDAL OGR, will find for each entry in CSV file the corresponding value from geotiff

Using location

Store it in new csv columns 

Using and running python script with arfuments as follow:

In [10]:
%run worldpop/load-wp.py --csv bigtable.csv --folder worldpop/GIS

## Data wrangling make date cyclic

Python script that will transform datetime feature into 2 cyclic values :

### One for the time of the day

Traffic is known to have an impact to PM25 pollution and hour of the day is related to traffic importance.

### If this is a weekday or not

Same remark about traffic

### A second one for the day in the year

To challenge and check if seasonality is correlated winter/sumer ?


Using and running python script with arfuments as follow:

In [11]:
%run cyclic/build-cyclic.py --csv bigtable.csv --column datetime

## Weather / meteo data extract

### weather data extraction made manually and storde locally

### Data wrangling done here below

In [12]:
df = pd.read_csv("bigtable.csv")
df['time'] = pd.to_datetime(df["datetime"],format="%Y-%m-%d %H:%M:%S %Z")
df['date'] = pd.Series(df.time).dt.strftime("%Y-%m-%d")
df['hour'] = pd.Series(df.time).dt.strftime("%H")

print("nb ground station measurement : {}".format(len(df)))


#2019-01-01T02:40:00
df_meteo = pd.read_csv("weather/weather.csv")
df_meteo['time'] = pd.to_datetime(df_meteo["DATE"],format="%Y-%m-%dT%H:%M:%S")
df_meteo['date'] = pd.Series(df_meteo.time).dt.strftime("%Y-%m-%d")
df_meteo['hour'] = pd.Series(df_meteo.time).dt.strftime("%H")
df_meteo['datehour'] = pd.Series(df_meteo.time).dt.strftime("%Y-%m-%d-%H")

print("nb meteo : {}".format(len(df_meteo)))

df_meteo.head()

    

nb ground station measurement : 243781
nb meteo : 14449


,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM,time,date,hour,datehour
0,2019-01-01T00:00:00,"+0089,1","999,9,C,0000,1","+0078,1","99999,9,9,N","006000,1,9,9","10165,1",2019-01-01 00:00:00,2019-01-01,00,2019-01-01-00
1,2019-01-01T00:11:00,"+0089,5","999,9,C,0000,5","+0078,5","00152,5,M,N","004023,5,N,5","99999,9",2019-01-01 00:11:00,2019-01-01,00,2019-01-01-00
2,2019-01-01T00:17:00,"+0089,5","999,9,C,0000,5","+0078,5","00122,5,M,N","002414,5,N,5","99999,9",2019-01-01 00:17:00,2019-01-01,00,2019-01-01-00
3,2019-01-01T00:22:00,"+0083,5","999,9,C,0000,5","+0067,5","00122,5,M,N","001207,5,N,5","99999,9",2019-01-01 00:22:00,2019-01-01,00,2019-01-01-00
4,2019-01-01T00:47:00,"+0080,5","999,9,C,0000,5","+0070,5","00122,5,M,N","001609,5,N,5","99999,9",2019-01-01 00:47:00,2019-01-01,00,2019-01-01-00


In [13]:
df_meteo=df_meteo.groupby(['datehour']).first().reset_index()

print("nb meteo unique hourly : {}".format(len(df_meteo)))

df_meteo.head()


nb meteo unique hourly : 8759


,datehour,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM,time,date,hour
0,2019-01-01-00,2019-01-01T00:00:00,"+0089,1","999,9,C,0000,1","+0078,1","99999,9,9,N","006000,1,9,9","10165,1",2019-01-01 00:00:00,2019-01-01,00
1,2019-01-01-01,2019-01-01T01:14:00,"+0089,5","999,9,C,0000,5","+0078,5","00091,5,M,N","003219,5,N,5","99999,9",2019-01-01 01:14:00,2019-01-01,01
2,2019-01-01-02,2019-01-01T02:10:00,"+0094,5","200,5,N,0015,5","+0083,5","00091,5,M,N","003219,5,N,5","99999,9",2019-01-01 02:10:00,2019-01-01,02
3,2019-01-01-03,2019-01-01T03:00:00,"+0100,1","220,1,N,0031,1","+0089,1","99999,9,9,N","004800,1,9,9","10127,1",2019-01-01 03:00:00,2019-01-01,03
4,2019-01-01-04,2019-01-01T04:52:00,"+0106,5","200,5,N,0026,5","+0094,5","00091,5,W,N","001609,5,N,5","10106,5",2019-01-01 04:52:00,2019-01-01,04


In [14]:
moments = df[['date', 'hour']].apply(tuple, axis=1).tolist()
moments = list(set(moments))
print("nb moments for stations measures : {}".format(len(moments)))


nb moments for stations measures : 29774


In [15]:
new_df = pd.merge(left=df, right=df_meteo, how='left', 
                  left_on=['date','hour'], right_on=['date','hour'])

print("nb ground station measurement after : {}".format(len(new_df)))
print("nb ground station measurement before : {}".format(len(df)))


new_df = new_df.drop(["time_x", "hour", "time_y"], axis=1)

print("***")

nb ground station measurement after : 243781
nb ground station measurement before : 243781
***


In [16]:
new_df.head()

,Unnamed: 0,datetime,pm25,type,sensor,station_id,x,y,Population,Dist-MRoads,...,date,24hr,datehour,DATE,Temperature,Wind,Dew,Sky,Visibility,ATM
0,0,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,...,2019-05-08,03:00:00,2019-05-08-15,2019-05-08T15:00:00,"+0211,1","050,1,N,0036,1","+0161,1","99999,9,9,N","016000,1,9,9","10218,1"
1,1,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,...,2019-05-08,04:00:00,2019-05-08-16,2019-05-08T16:52:00,"+0228,5","030,5,N,0041,5","+0167,5","00762,5,M,N","016093,5,N,5","10213,5"
2,2,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,...,2019-05-08,03:00:00,2019-05-08-15,2019-05-08T15:00:00,"+0211,1","050,1,N,0036,1","+0161,1","99999,9,9,N","016000,1,9,9","10218,1"
3,3,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,...,2019-05-08,04:00:00,2019-05-08-16,2019-05-08T16:52:00,"+0228,5","030,5,N,0041,5","+0167,5","00762,5,M,N","016093,5,N,5","10213,5"
4,4,2019-05-08 17:00:00 UTC,NaN,PurpleAir,B,14th & S ST NW B (undefined) (38.913805 -77.0...,38.913805,-77.03275,54.04969,0.072,...,2019-05-08,05:00:00,2019-05-08-17,2019-05-08T17:52:00,"+0228,5","060,5,N,0057,5","+0167,5","01006,5,M,N","016093,5,N,5","10211,5"


In [17]:
df =new_df

df["Temperature"]=pd.Series(df.Temperature).replace('\+', '', regex=True).replace(',', '.', regex=True).apply(pd.to_numeric, args=('coerce',))
df["Dew"]=pd.Series(df.Dew).replace('\+', '', regex=True).replace(',', '.', regex=True).apply(pd.to_numeric, args=('coerce',))
df["ATM"]=pd.Series(df.ATM).replace('\+', '', regex=True).replace(',', '.', regex=True).apply(pd.to_numeric, args=('coerce',))

In [21]:
# Simplify column names for stations
columns = {"Population":"population", "Dist-MRoads":"dist-mroads", "Dist-Setl":"dist-setl", "Dist-Coast":"dist-coast", "Dist-Forest":"dist-forest", "Slope":"slope", "Elevation":"elevation", "DATE":"date", "Temperature":"TEMP", "Wind":"WIND", "Dew":"DEW", "Sky":"SKY", "Visibility":"VIS"}
df = df.rename(columns=columns)
df = df.replace('14th & S ST NW  B (undefined) (38.913805 -77.03275) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '14th & S ST NW B')
df = df.replace('14th & S ST NW  (outside) (38.913805 -77.03275) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '14th & S ST NW A')
df = df.replace('V Street (outside) (38.918491 -77.037393) Primary 60_minute_average 01_01_2019 01_01_2020.csv', 'V Street')
df = df.replace('McMillan 1 (outside) (38.92185 -77.013271) Primary 60_minute_average 01_01_2019 01_01_2020.csv', 'McMillan 1')
df = df.replace('Courthouse  (outside) (38.88812 -77.088094) Primary 60_minute_average 01_01_2019 01_01_2020.csv', 'Courthouse')
df = df.replace('Cheverly (outside) (38.921633 -76.921768) Primary 60_minute_average 01_01_2019 01_01_2020-9.csv', 'Cheverly 9')
df = df.replace('Cheverly (outside) (38.921633 -76.921768) Primary 60_minute_average 01_01_2019 01_01_2020-1.csv', 'Cheverly 1')
df = df.replace('arlington (outside) (38.900099 -77.081078) Primary 60_minute_average 01_01_2019 01_01_2020-1.csv', 'Arlington')
df = df.replace('13th and E St SE B (undefined) (38.882974 -76.988037) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '13th & E ST SE B')
df = df.replace('13th and E St SE (outside) (38.882974 -76.988037) Primary 60_minute_average 01_01_2019 01_01_2020.csv', '13th & E ST SE A')



df.head()


,Unnamed: 0,datetime,pm25,type,sensor,station_id,x,y,population,dist-mroads,...,date,24hr,datehour,date,TEMP,WIND,DEW,SKY,VIS,ATM
0,0,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,...,2019-05-08,03:00:00,2019-05-08-15,2019-05-08T15:00:00,211.1,"050,1,N,0036,1",161.1,"99999,9,9,N","016000,1,9,9",10218.1
1,1,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,...,2019-05-08,04:00:00,2019-05-08-16,2019-05-08T16:52:00,228.5,"030,5,N,0041,5",167.5,"00762,5,M,N","016093,5,N,5",10213.5
2,2,2019-05-08 15:00:00 UTC,52.33,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,...,2019-05-08,03:00:00,2019-05-08-15,2019-05-08T15:00:00,211.1,"050,1,N,0036,1",161.1,"99999,9,9,N","016000,1,9,9",10218.1
3,3,2019-05-08 16:00:00 UTC,0.17,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,...,2019-05-08,04:00:00,2019-05-08-16,2019-05-08T16:52:00,228.5,"030,5,N,0041,5",167.5,"00762,5,M,N","016093,5,N,5",10213.5
4,4,2019-05-08 17:00:00 UTC,NaN,PurpleAir,B,14th & S ST NW B,38.913805,-77.03275,54.04969,0.072,...,2019-05-08,05:00:00,2019-05-08-17,2019-05-08T17:52:00,228.5,"060,5,N,0057,5",167.5,"01006,5,M,N","016093,5,N,5",10211.5


In [22]:

#Explain and transform data to numeric vetor or absolute value

# The rate of horizontal travel of air past a fixed point.
# MIN: 0000 MAX: 0900 UNITS: meters per second SCALING FACTOR: 10 
data = df['WIND'].str.split(pat=",", expand=True)
df['WIND'] = data[3].str.lstrip('0')

# The height above ground level (AGL) of the lowest cloud or obscuring phenomena layer aloft with 5/8 or more summation total sky cover, which may be predominantly opaque, or the vertical visibility into a surface-based obstruction. Unlimited = 22000.
# MIN: 00000 MAX: 22000 UNITS: Meters
data = df['SKY'].str.split(pat=",", expand=True)
df['SKY'] = data[0].str.lstrip('0')

# The horizontal distance at which an object can be seen and identified.
# MIN: 000000 MAX: 160000 UNITS: Meters
data = df['VIS'].str.split(pat=",", expand=True)
df['VIS'] = data[0].str.lstrip('0')



In [23]:
df.to_csv("bigtable.csv", index=False)